![image](car.jpeg)

**Car-ing is sharing**, an auto dealership company for car sales and rental, is taking their services to the next level thanks to **Large Language Models (LLMs)**.

As their newly recruited AI and NLP developer, you've been asked to prototype a chatbot app with multiple functionalities that not only assist customers but also provide support to human agents in the company.

The solution should receive textual prompts and use a variety of pre-trained Hugging Face LLMs to respond to a series of tasks, e.g. classifying the sentiment in a car’s text review, answering a customer question, summarizing or translating text, etc.


## Before you start

In order to complete the project you may wish to install some Hugging Face libraries such as `transformers` and `evaluate`.

In [14]:
!pip install transformers
!pip install evaluate==0.4.0
!pip install datasets==2.10.0
!pip install sentencepiece==0.1.97

from transformers import logging
logging.set_verbosity(logging.WARNING)

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


Importing required Libraries

In [15]:
# Start your code here!
from transformers import pipeline 
import pandas as pd
from datasets import load_dataset

from torch.utils.data import DataLoader
import evaluate

Sentiment analysis using transformer library

In [16]:
from transformers import pipeline
from datasets import load_dataset

model_name = 'distilbert-base-uncased-finetuned-sst-2-english'

model = pipeline('sentiment-analysis', model=model_name)

dataset = pd.read_csv('data/car_reviews.csv', delimiter=';')
dataset

,Review,Class
0,I am very satisfied with my 2014 Nissan NV SL....,POSITIVE
1,The car is fine. It's a bit loud and not very ...,NEGATIVE
2,"My first foreign car. Love it, I would buy ano...",POSITIVE
3,I've come across numerous reviews praising the...,NEGATIVE
4,I've been dreaming of owning an SUV for quite ...,POSITIVE


Viewing predictions

In [17]:
reviews = dataset['Review'].to_list()
true_labels = dataset['Class'].to_list()

predicted_labels = model(reviews)

for review, label, prediction in zip(reviews, true_labels, predicted_labels):
    print(f"Review: {review} \nActual Sentiment Text: {label} \n Prediction: {prediction}")

Review: I am very satisfied with my 2014 Nissan NV SL. I use this van for my business deliveries and personal use. Camping, road trips, etc. We dont have any children so I store most of the seats in my warehouse. I wanted the passenger van for the rear air conditioning. We drove our van from Florida to California for a Cross Country trip in 2014. We averaged about 18 mpg. We drove thru a lot of rain and It was a very comfortable and stable vehicle. The V8 Nissan Titan engine is a 500k mile engine. It has been tested many times by delivery and trucking companies. This is why Nissan gives you a 5 year or 100k mile bumper to bumper warranty. Many people are scared about driving this van because of its size. But with front and rear sonar sensors, large mirrors and the back up camera. It is easy to drive. The front and rear sensors also monitor the front and rear sides of the bumpers making it easier to park close to objects. Our Nissan NV is a Tow Monster. It pulls our 5000 pound travel tr

reviews[1]

In [18]:
reviews[1]

"The car is fine. It's a bit loud and not very powerful. On one hand, compared to its peers, the interior is well-built. The transmission failed a few years ago, and the dealer replaced it under warranty with no issues. Now, about 60k miles later, the transmission is failing again. It sounds like a truck, and the issues are well-documented. The dealer tells me it is normal, refusing to do anything to resolve the issue. After owning the car for 4 years, there are many other vehicles I would purchase over this one. Initially, I really liked what the brand is about: ride quality, reliability, etc. But I will not purchase another one. Despite these concerns, I must say, the level of comfort in the car has always been satisfactory, but not worth the rest of issues found."

In [19]:
predictions = [ 1 if prediction['label'] == 'POSITIVE' else 0 for prediction in predicted_labels]
print(predictions)

true_labels
real_labels = [1 if label == 'POSITIVE' else 0 for label in true_labels]
real_labels

[1, 1, 1, 0, 1]


[1, 0, 1, 0, 1]

Evaluating using the evaluate library and the Accuracy and F1 metrics

In [20]:
accuracy = evaluate.load('accuracy')
f1  = evaluate.load('f1')


accuracy_result_dict = accuracy.compute(references=real_labels, predictions=predictions)
accuracy_result = accuracy_result_dict['accuracy']
f1_result_dict = f1.compute(references=real_labels, predictions=predictions)
f1_result = f1_result_dict['f1']
print(f"Accuracy result: {accuracy_result}")
print(f"F1 result: {f1_result}")

Accuracy result: 0.8
F1 result: 0.8571428571428571


Performing Translation Task

In [21]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# model_name = "Helsinki-NLP/opus-mt-en-es"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

first_review = reviews[0]
translator = pipeline('translation', model="Helsinki-NLP/opus-mt-en-es")
translated_review = translator(first_review, max_length=27)[0]['translation_text']
print(f"Model output: \n{translated_review}")

Your input_length: 365 is bigger than 0.9 * max_length: 27. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


Model output: 
Estoy muy satisfecho con mi 2014 Nissan NV SL. Uso esta furgoneta para mis entregas de negocios y uso personal.


Evaluating the translation output using bleu score

In [22]:
bleu = evaluate.load('bleu')

with open("data/reference_translations.txt", 'r') as file:
    lines = file.readlines()
references = [line.strip() for line in lines]
print(f"Spanish references: \n {references}")

bleu_score = bleu.compute(predictions=[translated_review], references=[references])
print(f"Bleu Score: \n {bleu_score['bleu']}")

Spanish references: 
 ['Estoy muy satisfecho con mi Nissan NV SL 2014. Utilizo esta camioneta para mis entregas comerciales y uso personal.', 'Estoy muy satisfecho con mi Nissan NV SL 2014. Uso esta furgoneta para mis entregas comerciales y uso personal.']
Bleu Score: 
 0.6022774485691839


Question Answering Task

In [23]:
from  transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch
question_model = "deepset/minilm-uncased-squad2"
question = "What did he like about the brand?"

tokenizer = AutoTokenizer.from_pretrained(question_model)

context = reviews[1]
inputs = tokenizer(question, context, return_tensors='pt')
model = AutoModelForQuestionAnswering.from_pretrained(question_model)
with torch.no_grad():
    outputs = model(**inputs)
    
start_idx = torch.argmax(outputs.start_logits)
end_idx = torch.argmax(outputs.end_logits) + 1

In [24]:
answer_span = inputs['input_ids'][0][start_idx:end_idx]
answer = tokenizer.decode(answer_span)                                  
print(f"Answer: {answer}")                          

Answer: ride quality, reliability


Summarization Task

In [25]:
last_review = reviews[-1]

summary_ml = 't5-small'
summarizer = pipeline('summarization', model=summary_ml)

outputs = summarizer(last_review, max_length=54)

summarized_text = outputs[0]['summary_text']

print(f"Summarized text: \n{summarized_text}")

Summarized text: 
the Nissan Rogue provides me with the desired SUV experience without burdening me with an exorbitant payment . the financial arrangement is quite reasonable . I find myself needing extra caution when making lane changes .
